In [1]:
%pip install --quiet ipython-extensions
!qbraid jobs disable qbraid_sdk
%pip install --quiet --upgrade pip
%pip install --quiet amazon-braket-sdk==1.37.0
%pip install --quiet --upgrade amazon-braket-schemas
%pip install --quiet qbraid

Note: you may need to restart the kernel to use updated packages.
Disable successful. You are now submitting quantum jobs with your own AWS credentials.

To re-enable, run: `qbraid jobs enable qbraid_sdk`
Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
qbraid 0.4.5 requires amazon-braket-sdk<1.60.0,>=1.42.1, but you have amazon-braket-sdk 1.37.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# from qbraid.providers.aws import BraketDevice, BraketProvider
from braket.aws import AwsDevice
from qiskit import QuantumCircuit
from braket.aws import AwsQuantumJob
import importlib
import braket.aws

# from qiskit_braket_provider import AWSBraketProvider, BraketLocalBackend

In [3]:
import cirq
import numpy as np
import matplotlib.pyplot as plt

def prep(sys,state):
    for i in range(len(state)):
        if state[i]=='1':
            yield cirq.X(sys[i])

def zz_pump(sys,env,p,state):
    yield from prep(sys,state)
    yield cirq.CNOT(sys[1],sys[0])
    yield cirq.X(env[0])
    yield cirq.CNOT(sys[0],env[0])
    yield cirq.Rx(rads=np.arccos(1-2*p)).on(sys[0]).controlled_by(env[0])
    yield cirq.CNOT(sys[0],env[0])
    yield cirq.H(sys[1])

def xx_pump(sys,env,p,state):
    yield from prep(sys,state)
    yield cirq.CNOT(sys[1],sys[0])
    yield cirq.H(sys[1])
    yield cirq.X(env[0])
    yield cirq.CNOT(sys[1],env[0])
    yield cirq.rx(rads=np.arccos(1-2*p)).on(sys[1]).controlled_by(env[0])
    yield cirq.CNOT(sys[1],env[0])

def zz_xx_pump(sys,env,p,state):
    yield from prep(sys,state)
    yield cirq.CNOT(sys[1],sys[0])
    yield cirq.X(env[0])
    yield cirq.CNOT(sys[0],env[0])
    yield cirq.rx(rads=np.arccos(1-2*p)).on(sys[0]).controlled_by(env[0])
    yield cirq.CNOT(sys[0],env[0])
    yield cirq.H(sys[1])
    yield cirq.X(env[1])
    yield cirq.CNOT(sys[1],env[1])
    yield cirq.rx(rads=np.arccos(1-2*p)).on(sys[1]).controlled_by(env[1])
    yield cirq.CNOT(sys[1],env[1])


In [ ]:
probs = np.linspace(0,1,num=5)
repcnt = 960

In [3]:
!qbraid jobs enable qbraid_sdk

Successfully enabled qBraid Quantum Jobs in the qbraid_sdk environment.
Every AWS job you run will now be submitted through the qBraid API, so no access keys/tokens are necessary. 

To disable, run: `qbraid jobs disable qbraid_sdk`


In [4]:
from qbraid import get_devices
get_devices()

Provider,Name,qBraid ID,Status
AWS,DM1,aws_dm_sim,●
AWS,SV1,aws_sv_sim,●
AWS,TN1,aws_tn_sim,●
IBM,Brisbane,ibm_q_brisbane,●
IBM,Ext. stabilizer simulator,ibm_q_simulator_extended_stabilizer,●
IBM,Kyoto,ibm_q_kyoto,●
IBM,MPS simulator,ibm_q_simulator_mps,●
IBM,Osaka,ibm_q_osaka,●
IBM,Oslo,ibm_q_oslo,○
IBM,QASM simulator,ibm_q_qasm_simulator,●


In [5]:
from qbraid import device_wrapper, job_wrapper, get_jobs
# ionq_id = "aws_ionq_aria2"
# qbraid_aws_device = device_wrapper(ionq_id)
# qbraid_aws_device.vendor_dlo
# qbraid_aws_device.info

qbraid_id = 'aws_rigetti_aspen_m3'
qdevice = device_wrapper(qbraid_id)

ClientError: An error occurred (UnrecognizedClientException) when calling the GetDevice operation: The security token included in the request is invalid.

In [7]:
# print(qbraid_aws_device.vendor_dlo)
# %pip install --quiet qbraid.providers.aws

In [8]:
# from qbraid.providers.aws import BraketDevice, BraketProvider
from qiskit import QuantumCircuit

circuit = QuantumCircuit(2)
circuit.h(0)
circuit.cx(0, 1)

qjob = qdevice.run(circuit)
type(qjob)

# provider = BraketProvider()

# arn = "arn:aws:braket:eu-west-2::device/qpu/oqc/Lucy"
# arn = "arn:aws:braket:::device/quantum-simulator/amazon/sv1"

# aws_device = provider.get_device("arn:aws:braket:eu-west-2::device/qpu/oqc/Lucy")

# qbraid_device = BraketDevice(aws_device)

# job = qbraid_device.run(circuit, shots=1)

# job.status()


NameError: name 'qdevice' is not defined

In [1]:
def batch_3(p):
    env = [cirq.LineQubit(0),cirq.LineQubit(3),cirq.LineQubit(6)]
    sys = [cirq.LineQubit(1),cirq.LineQubit(2),cirq.LineQubit(4),cirq.LineQubit(5),cirq.LineQubit(7),cirq.LineQubit(8)]
    circ = cirq.Circuit()
    circ.append(zz_pump(sys[0:2],env[0:1],p,'10'))
    circ.append(zz_pump(sys[2:4],env[1:2],p,'11'))
    circ.append(xx_pump(sys[4:6],env[2:3],p,'00'))
    qbraid_aws_job = qbraid_aws_device.run(circ, shots=repcnt)
    return qbraid_aws_job

def batch_4(p):
    env = [cirq.LineQubit(0),cirq.LineQubit(3),cirq.LineQubit(6)]
    sys = [cirq.LineQubit(1),cirq.LineQubit(2),cirq.LineQubit(4),cirq.LineQubit(5),cirq.LineQubit(7),cirq.LineQubit(8)]
    circ = cirq.Circuit()
    circ.append(xx_pump(sys[0:2],env[0:1],p,'01'))
    circ.append(xx_pump(sys[2:4],env[1:2],p,'10'))
    circ.append(xx_pump(sys[4:6],env[2:3],p,'11'))
    qbraid_aws_job = qbraid_aws_device.run(circ, shots=repcnt)
    return qbraid_aws_job
    

In [2]:
# qjob_3_p0= batch_3(probs[0])

In [3]:
# qjob_4_p0 = batch_4(probs[0])

In [5]:
from qbraid import get_jobs
get_jobs(filters={"qbraidDeviceId": "aws_ionq"})

In [ ]:
from qbraid import job_wrapper
saved_job_id = 'aws_sv_sim-jessjohnbritto-qjob-u8nzibwhq9o7g5ga2fic'
qjob = job_wrapper(saved_job_id)
qresult = qjob.result()
qresult.measurement_counts()

In [ ]:
resultsdict_3_p0 = {}
resultsdict_4_p0 = {}

In [5]:
## Batch-3 and 4 - XX, ZZ10 and ZZ11
zz11p0,zz10p0,xx00p0,xx01p0,xx10p0,xx11p0 = Counter(),Counter(),Counter(),Counter(),Counter(),Counter()

def resul_analy(resultsdict_3_p0, resultsdict_4_p0):
    for key,cnt in resultsdict_1_p0.items():
        k1 = int(key[0:2],base=2)
        k2 = int(key[2:4],base=2)
        k3 = int(key[4:6],base=2)
        zz10p0[k1]+=cnt
        zz11p0[k2]+=cnt
        xx00p0[k3]+=cnt

    for key,cnt in resultsdict_2_p0.items():
        k1 = int(key[0:2],base=2)
        k2 = int(key[2:4],base=2)
        k3 = int(key[4:6],base=2)
        xx01p0[k1]+=cnt
        xx10p0[k2]+=cnt
        xx11p0[k3]+=cnt
    
    xxp0 = (xx00p0+xx01p0+xx10p0+xx11p0)
    for key,cnt in xxp0.items():
        xxp0[key] = cnt/(4*repcnt)
    return xxp0, zz10p0, zz11p0

NameError: name 'Counter' is not defined

In [ ]:
# qjob_3_p1= batch_3(probs[1])

In [ ]:
# qjob_4_p1=batch_4(probs[1])

In [ ]:
from qbraid import get_jobs
get_jobs(filters={"qbraidDeviceId": "aws_ionq"})

In [ ]:
from qbraid import job_wrapper
saved_job_id = 'aws_sv_sim-jessjohnbritto-qjob-u8nzibwhq9o7g5ga2fic'
qjob = job_wrapper(saved_job_id)
qresult = qjob.result()
qresult.measurement_counts()

In [ ]:
resultsdict_3_p1 = {}
resultsdict_4_p1 = {}

In [ ]:
## Batch-3 and 4 - XX, ZZ10 and ZZ11
zz11p1,zz10p1,xx00p1,xx01p1,xx10p1,xx11p1 = Counter(),Counter(),Counter(),Counter(),Counter(),Counter()
xxp1, zz10p1, zz11p1 = resul_analy(resultsdict_3_p1, resultsdict_4_p1)

In [ ]:
# qjob_3_p2=batch_3(probs[2])

In [ ]:
# qjob_4_p2=batch_4(probs[2])

In [ ]:
from qbraid import get_jobs
get_jobs(filters={"qbraidDeviceId": "aws_ionq"})

In [ ]:
from qbraid import job_wrapper
saved_job_id = 'aws_sv_sim-jessjohnbritto-qjob-u8nzibwhq9o7g5ga2fic'
qjob = job_wrapper(saved_job_id)
qresult = qjob.result()
qresult.measurement_counts()

In [ ]:
resultsdict_3_p2 = {}
resultsdict_4_p2 = {}

In [ ]:
## Batch-3 and 4 - XX, ZZ10 and ZZ11
zz11p2,zz10p2,xx00p2,xx01p2,xx10p2,xx11p2 = Counter(),Counter(),Counter(),Counter(),Counter(),Counter()
xxp2, zz10p2, zz11p2 = resul_analy(resultsdict_3_p2, resultsdict_4_p2)

In [ ]:
# qjob_3_p3=batch_3(probs[3])

In [ ]:
# qjob_4_p3=batch_4(probs[3])

In [ ]:
from qbraid import get_jobs
get_jobs(filters={"qbraidDeviceId": "aws_ionq"})

In [ ]:
from qbraid import job_wrapper
saved_job_id = 'aws_sv_sim-jessjohnbritto-qjob-u8nzibwhq9o7g5ga2fic'
qjob = job_wrapper(saved_job_id)
qresult = qjob.result()
qresult.measurement_counts()

In [ ]:
resultsdict_3_p3 = {}
resultsdict_4_p3 = {}

In [ ]:
## Batch-3 and 4 - XX, ZZ10 and ZZ11
zz11p3,zz10p3,xx00p3,xx01p3,xx10p3,xx11p3 = Counter(),Counter(),Counter(),Counter(),Counter(),Counter()
xxp3, zz10p3, zz11p3 = resul_analy(resultsdict_3_p3, resultsdict_4_p3)

In [ ]:
# qjob_3_p4=batch_3(probs[4])

In [ ]:
# qjob_4_p4=batch_4(probs[4])

In [ ]:
from qbraid import get_jobs
get_jobs(filters={"qbraidDeviceId": "aws_ionq"})

In [ ]:
from qbraid import job_wrapper
saved_job_id = 'aws_sv_sim-jessjohnbritto-qjob-u8nzibwhq9o7g5ga2fic'
qjob = job_wrapper(saved_job_id)
qresult = qjob.result()
qresult.measurement_counts()

In [ ]:
resultsdict_3_p4 = {}
resultsdict_4_p4 = {}

In [ ]:
## Batch-3 and 4 - XX, ZZ10 and ZZ11
zz11p4,zz10p4,xx00p4,xx01p4,xx10p4,xx11p4 = Counter(),Counter(),Counter(),Counter(),Counter(),Counter()
xxp4, zz10p4, zz11p4 = resul_analy(resultsdict_3_p4, resultsdict_4_p4)

In [ ]:
# ## Batch-1 and 2 - ZZXX, ZZ00 and ZZ01
# zz00p0,zz01p0,zzxx00p0,zzxx01p0,zzxx10p0,zzxx11p0 = Counter(),Counter(),Counter(),Counter(),Counter(),Counter()

# for key,cnt in resultsdict_1_p0.items():
#     k1 = int(key[0:2],base=2)
#     k2 = int(key[2:4],base=2)
#     k3 = int(key[4:5],base=2)
#     zzxx00p0[k1]+=cnt
#     zzxx01p0[k2]+=cnt
#     zz00p0[k3]+=cnt

# for key,cont in resultsdict_2_p0.items():
#     k1 = int(key[0:2],base=2)
#     k2 = int(key[2:4],base=2)
#     k3 = int(key[4:5],base=2)
#     zzxx10p0[k1]+=cnt
#     zzxx11p0[k2]+=cnt
#     zz01p0[k3]+=cnt
    
# zzxxp0 = (zzxx00p0+zzxx01p0+zzxx10p0+zzxx11p0)
# for key,cnt in zzxxp0.items():
#     zzxxp0[key] = cnt/(4*repcnt)
# print(zzxxp0)